# Overall Accuracy

In [10]:
import os
import pandas as pd

base_dir = "/Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results"
output_csv = os.path.join(base_dir, "summary_accuracy.csv")

rows = []

for model_name in os.listdir(base_dir):
    model_path = os.path.join(base_dir, model_name)
    if not os.path.isdir(model_path):
        continue

    for file_name in os.listdir(model_path):
        if not file_name.endswith(".tsv"):
            continue

        file_path = os.path.join(model_path, file_name)

        parts = file_name.replace("_results_bycond.tsv", "").split("_")
        if len(parts) == 2:
            train_lang, test_lang = parts
        else:
            train_lang, test_lang = "unknown", "unknown"

        df = pd.read_csv(file_path, sep="\t")

        if  "accuracy" not in df.columns:
            print(f"Skipping {file_path}, missing expected columns.")
            continue

        accuracy = df["accuracy"].iloc[0]

        rows.append({
            "Model": model_name,
            "Train_Lang": train_lang,
            "Test_Lang": test_lang,
            "Accuracy": accuracy
        })

summary_df = pd.DataFrame(rows)

summary_df.to_csv(output_csv, index=False)

print(f"Summary table saved to {output_csv}")
print(summary_df)

# To Latex table
input_csv = output_csv
output_txt = os.path.join(base_dir, "summary_accuracy_table.txt")

df = pd.read_csv(input_csv)

train_lang_labels = {
    "fon": "Fon",
    "all": "Combined four languages",
}

test_langs = sorted(df["Test_Lang"].unique())

models = df["Model"].unique()

lines = []
header = "Model & Train Data & " + " & ".join(test_langs) + " \\\\"
lines.append(header)
lines.append("\\hline")

for model in models:
    model_df = df[df["Model"] == model]
    
    train_conditions = [
        ("target", "Target language"),
        ("fon", train_lang_labels.get("fon")),
        ("all", train_lang_labels.get("all"))
    ]

    for i, (cond_key, display_train) in enumerate(train_conditions):
        if cond_key == "target":
            # Select rows where train_lang == test_lang
            sub_df = model_df[model_df["Train_Lang"] == model_df["Test_Lang"]]
        else:
            sub_df = model_df[model_df["Train_Lang"] == cond_key]

        if display_train is None:
            display_train = cond_key.capitalize()

        accuracies = []
        for test_lang in test_langs:
            match = sub_df[sub_df["Test_Lang"] == test_lang]
            if not match.empty:
                accuracy = match["Accuracy"].values[0] * 100
                accuracies.append(f"{accuracy:.2f}\%")
            else:
                accuracies.append("-")

        model_display = model if i == 0 else ""
        line = f"{model_display:<15} & {display_train:<30} & " + " & ".join(accuracies) + " \\\\"
        lines.append(line)

with open(output_txt, "w") as f:
    f.write("\n".join(lines))

print(f"\n LaTeX table saved to {output_txt}")

Skipping /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/XLM-R/fon_ibo_results_bytokentype.tsv, missing expected columns.
Skipping /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/XLM-R/fon_kin_results_bytokentype.tsv, missing expected columns.
Skipping /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/XLM-R/fon_sna_results_bytokentype.tsv, missing expected columns.
Skipping /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/XLM-R/ibo_ibo_results_bytokentype.tsv, missing expected columns.
Skipping /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/XLM-R/kin_kin_results_bytokentype.tsv, missing expected columns.
Skipping /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/XLM-R/all_ibo_results_bytokentype.tsv, missing expected columns.
Skipping /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/XLM-R/sna_sna_results_

# Other metrics

In [2]:
import os
import pandas as pd

base_dir = "/Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results"

rows = []

for model_name in os.listdir(base_dir):
    model_path = os.path.join(base_dir, model_name)
    if not os.path.isdir(model_path):
        continue

    for file_name in os.listdir(model_path):
        if not file_name.endswith(".tsv"):
            continue

        file_path = os.path.join(model_path, file_name)

        parts = file_name.replace("_results_bycond.tsv", "").split("_")
        if len(parts) == 2:
            train_lang, test_lang = parts
        else:
            continue

        df = pd.read_csv(file_path, sep="\t")

        df["Model"] = model_name
        df["Train_Lang"] = train_lang
        df["Test_Lang"] = test_lang

        rows.append(df)

combined_df = pd.concat(rows, ignore_index=True)

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

metrics = ["macro-precision", "macro-recall", "macro-f1"]

output_fig_dir = os.path.join(base_dir, "figures")
os.makedirs(output_fig_dir, exist_ok=True)

# Define the desired order for the bars
desired_order = [
    "ibo→ibo", "kin→kin", "sna→sna",
    "fon→ibo", "fon→kin", "fon→sna",
    "all→ibo", "all→kin", "all→sna"
]

combined_df["Train→Test"] = combined_df["Train_Lang"] + "→" + combined_df["Test_Lang"]

combined_df["Train→Test"] = pd.Categorical(
    combined_df["Train→Test"],
    categories=desired_order,
    ordered=True
)

sns.set(style="whitegrid", context="talk", font_scale=1.2)

for metric in metrics:
    plt.figure(figsize=(14, 7))
    
    ax = sns.barplot(
        data=combined_df,
        x="Train→Test",
        y=metric,
        hue="Model",
        ci=None,
        palette="Set2"
    )
    
    ax.set_title(f"{metric} for all Train/Test pairs", fontsize=18, weight='bold')
    ax.set_xlabel("Train → Test language", fontsize=14)
    ax.set_ylabel(metric.replace("-", " ").title(), fontsize=14)
    
    plt.xticks(rotation=45, ha="right", fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(title="Model", fontsize=12, title_fontsize=13, loc="upper right")
    plt.ylim(0, 1)
    plt.tight_layout()
    fig_path = os.path.join(output_fig_dir, f"{metric}.png")
    plt.savefig(fig_path, dpi=300)
    print(f"Saved figure: {fig_path}")
    plt.close()


/var/folders/nf/r2gt2wsd5pnb2cqvrbrw75t40000gn/T/ipykernel_33541/1204666713.py:29: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  ax = sns.barplot(
/var/folders/nf/r2gt2wsd5pnb2cqvrbrw75t40000gn/T/ipykernel_33541/1204666713.py:29: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  ax = sns.barplot(


Saved figure: /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/figures/macro-precision.png
Saved figure: /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/figures/macro-recall.png
Saved figure: /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/figures/macro-f1.png


/var/folders/nf/r2gt2wsd5pnb2cqvrbrw75t40000gn/T/ipykernel_33541/1204666713.py:29: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  ax = sns.barplot(


In [9]:
import os
import pandas as pd

base_dir = "/Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results"
output_csv = os.path.join(base_dir, "summary_bytokentype.csv")
output_tex = os.path.join(base_dir, "summary_bytokentype_table.txt")

rows = []

for model_name in os.listdir(base_dir):
    model_path = os.path.join(base_dir, model_name)
    if not os.path.isdir(model_path):
        continue

    for file_name in os.listdir(model_path):
        if not file_name.endswith("_results_bytokentype.tsv"):
            continue

        file_path = os.path.join(model_path, file_name)

        parts = file_name.replace("_results_bytokentype.tsv", "").split("_")
        if len(parts) == 2:
            train_lang, test_lang = parts
        else:
            continue

        df = pd.read_csv(file_path, sep="\t")

        if not {"target_class", "precision", "recall", "f1"}.issubset(df.columns):
            print(f"Skipping {file_path}, missing expected columns.")
            continue

        for _, row in df.iterrows():
            rows.append({
                "Model": model_name,
                "Train_Lang": train_lang,
                "Test_Lang": test_lang,
                "Target_Class": row["target_class"],
                "Precision": row["precision"],
                "Recall": row["recall"],
                "F1": row["f1"]
            })

summary_df = pd.DataFrame(rows)
summary_df.to_csv(output_csv, index=False)
print(f"Saved combined results to {output_csv}")

models = sorted(summary_df["Model"].unique())
token_classes = sorted(summary_df["Target_Class"].unique())

lines = []
header_parts = ["Token Class"]
for model in models:
    header_parts.extend([f"{model} Prec.", f"{model} Rec.", f"{model} F1"])
header = " & ".join(header_parts) + " \\\\"
lines.append(header)
lines.append("\\hline")

for token in token_classes:
    row_parts = [token]
    for model in models:
        sub_df = summary_df[(summary_df["Model"] == model) &
                            (summary_df["Target_Class"] == token)]
        if sub_df.empty:
            row_parts.extend(["-", "-", "-"])
        else:
            precision = sub_df["Precision"].mean()
            recall = sub_df["Recall"].mean()
            f1 = sub_df["F1"].mean()
            row_parts.extend([f"{precision:.4f}", f"{recall:.4f}", f"{f1:.4f}"])
    line = " & ".join(row_parts) + " \\\\"
    lines.append(line)

with open(output_tex, "w") as f:
    f.write("\n".join(lines))

print(f"LaTeX table saved to {output_tex}")

Saved combined results to /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/summary_bytokentype.csv
LaTeX table saved to /Users/huxiyan/Desktop/H/25Fall/COSC426/Dione-Midterm-Replication-/results/summary_bytokentype_table.txt


In [13]:
output_dir = os.path.join(base_dir, "by_testlang_tables")
os.makedirs(output_dir, exist_ok=True)

for model in ["afrolm", "xlm-r"]:
    model_df = summary_df[summary_df["Model"].str.lower() == model]
    if model_df.empty:
        print(f"No data found for model {model}, skipping.")
        continue

    pivot_df = model_df.groupby(["Test_Lang", "Target_Class"])["F1"].mean().reset_index()
    pivot_df = pivot_df.pivot(index="Test_Lang", columns="Target_Class", values="F1").fillna("-")

    token_classes = list(pivot_df.columns)
    lines = []

    header = "Test Lang & " + " & ".join(token_classes) + " \\\\"
    lines.append(header)
    lines.append("\\hline")

    for test_lang, row in pivot_df.iterrows():
        values = []
        for token in token_classes:
            val = row[token]
            if isinstance(val, str):
                values.append("-")
            else:
                values.append(f"{val:.4f}")
        line = f"{test_lang} & " + " & ".join(values) + " \\\\"
        lines.append(line)

    output_tex_model = os.path.join(output_dir, f"{model}_by_testlang_f1_table.txt")
    with open(output_tex_model, "w") as f:
        f.write("\n".join(lines))

    print(f"LaTeX F1 table by Test_Lang saved for {model}")

LaTeX F1 table by Test_Lang saved for afrolm
LaTeX F1 table by Test_Lang saved for xlm-r
